In [44]:
import pandas as pd
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import dataframe_image as dfi
import pylatex as pl
from pylatex import Command, Document, Section, Subsection
from pylatex.utils import NoEscape, italic
from mdutils.mdutils import *
from mdutils import *
import os

In [80]:
wheel="blitz"
brand="Begode"
year="2024"
battery_cap=None

In [81]:
portraits = os.listdir('imgs/wheels/portrait')
portrait = [item for item in portraits if item.startswith(wheel) ][0]

wheel_specs = pd.read_csv("specs/"+wheel+".csv",sep='|')

wheel_specs_tab=wheel_specs.values.copy()
for i in range(0,len(wheel_specs_tab)):
   wheel_specs_tab[i]=["**"+wheel_specs_tab[i][0]+"**",wheel_specs_tab[i][1]]
wheel_specs_tab = ['**Field**','**Data**']+[item for row in wheel_specs_tab for item in row]

battery_cap=wheel_specs[wheel_specs["Field"]=="Battery"]
battery_cap=battery_cap.values
battery_cap=battery_cap[0][1].split("Wh")[0]
battery_cap=int(battery_cap)/1000

# STATS
## ENV

In [82]:
stats_dir="stats/"+wheel
analysis_dir='analysis/'+wheel

env_stats=pd.read_csv(stats_dir+'/env.csv')
#env_stats.columns
env=env_stats[['trip_distance_km','rider_weight_kg','tire_pressure_bars','outdoor_temperature_c','altitude_difference']]
env=env.describe()

env_stats['count']=1
trips=env_stats[['trip_type','count']].groupby('trip_type').count()
apps=env_stats[['data_origin','count']].groupby('data_origin').count()

env=env.style.background_gradient()
env


,trip_distance_km,rider_weight_kg,tire_pressure_bars,outdoor_temperature_c,altitude_difference
count,2.000000,2.000000,2.000000,2.000000,2.000000
mean,34.138000,85.000000,2.500000,27.500000,109.600000
std,21.381495,0.000000,0.000000,3.535534,13.152186
min,19.019000,85.000000,2.500000,25.000000,100.300000
25%,26.578500,85.000000,2.500000,26.250000,104.950000
50%,34.138000,85.000000,2.500000,27.500000,109.600000
75%,41.697500,85.000000,2.500000,28.750000,114.250000
max,49.257000,85.000000,2.500000,30.000000,118.900000


# Conso

In [83]:
conso_stats=pd.read_csv(stats_dir+'/conso.csv')
conso=conso_stats.describe()
estimated_range = int(battery_cap/conso[['average_wh/km']].values[1][0]*1000)
wheel_specs_tab +=['**Estimated range (baeed on data)**',str(estimated_range)+" km"]

In [84]:
conso=conso.style.background_gradient()
conso

,trip_id,initial_battery_kwh,end_batt_lvl_pct,end_batt_voltage,estimated_end_batt_lvl_pct,conso_kwh,regen_kwh,conso_corrected_kwh,average_wh/km,voltage_sag_coeff
count,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000
mean,0.500000,2.400000,41.000000,116.900000,45.000000,1.390000,-0.075000,1.315000,39.065000,-0.231500
std,0.707107,0.000000,35.355339,9.333810,28.284271,0.735391,0.063640,0.671751,5.621499,0.041719
min,0.000000,2.400000,16.000000,110.300000,25.000000,0.870000,-0.120000,0.840000,35.090000,-0.261000
25%,0.250000,2.400000,28.500000,113.600000,35.000000,1.130000,-0.097500,1.077500,37.077500,-0.246250
50%,0.500000,2.400000,41.000000,116.900000,45.000000,1.390000,-0.075000,1.315000,39.065000,-0.231500
75%,0.750000,2.400000,53.500000,120.200000,55.000000,1.650000,-0.052500,1.552500,41.052500,-0.216750
max,1.000000,2.400000,66.000000,123.500000,65.000000,1.910000,-0.030000,1.790000,43.040000,-0.202000


# Summaries

In [85]:
sum_stats=pd.read_csv(stats_dir+'/summary.csv')
sum_stats=sum_stats.drop(['trip_id'],axis=1)

dfs={}

cols = sum_stats.columns[1:-1]
for c in cols:
    df=sum_stats[['stat',c]].groupby('stat').describe()
    dfs.update({c:df})

In [86]:

mdFile = MdUtils(file_name="analysis/"+wheel,title='')
# Additional Markdown syntax...

mdFile.new_header(level=1, title=brand+" " + wheel.replace("_"," ").upper())
mdFile.write('**RELEASE YEAR :**  '+year)

mdFile.new_header(level=2, title="Portrait")
mdFile.new_paragraph(Html.image(path="../imgs/wheels/portrait/"+portrait, size='200'))
mdFile.new_line()

mdFile.new_header(level=2, title="Specifications")
mdFile.new_line()
mdFile.new_table(columns=2, rows=21, text=wheel_specs_tab, text_align='center')

mdFile.new_header(level=1, title="Usage Statistcs")

mdFile.new_header(level=2, title="Number of trips per source")
mdFile.write(apps.to_html())

mdFile.new_header(level=2, title="Number of trips per type")
mdFile.write(trips.to_html())

mdFile.new_header(level=2, title="Trips environment statistics")
mdFile.write(env.to_html())

mdFile.new_header(level=2, title="Trips energy statistics")
mdFile.write(conso.to_html())

mdFile.new_header(level=2, title="Trips statistics per field")

for d in dfs:
    field=dfs.get(d)
    field=field.style.background_gradient()
    mdFile.write(field.to_html())
    mdFile.new_line()
    mdFile.write('---')
    mdFile.new_line()
    mdFile.new_line()

mdFile.new_header(level=2, title="Trips details")
trips_links=[]
anal_files=os.listdir(analysis_dir)
anal_files=[file for file in anal_files if file.endswith('.md')]
for a in anal_files:
    trip_id=a.split('_')[-1]
    trip_id=trip_id.split('.')[0]
    trips_links+=['Trip '+trip_id,'[Details]'+'('+wheel+'/'+a+')']
trips_links=['**Trip id**','**link**']+trips_links

mdFile.new_table(columns=2, rows=len(anal_files)+1, text=trips_links, text_align='center')

mdFile.create_md_file()

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



In [118]:
anal_files

['z10_0.md']